In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df_org = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-01-26/plastics.csv')

In [ ]:
cont = pd.read_csv('continents2.csv')

# Add Continents

In [ ]:
cont.head()
cont_simple = cont[['name', 'region']]
cont_simple.head()


,name,region
0,Afghanistan,Asia
1,Åland Islands,Europe
2,Albania,Europe
3,Algeria,Africa
4,American Samoa,Oceania


In [ ]:
cont_simple = cont_simple.replace('United States', 'United States of America')
cont_simple = cont_simple.replace('Korea, Republic of', 'Korea')

In [ ]:
df_org = df_org.replace('Cote D_ivoire', "Côte D'Ivoire")
df_org = df_org.replace('ECUADOR', 'Ecuador')
df_org = df_org.replace('NIGERIA', 'Nigeria')
df_org = df_org.replace('Taiwan_ Republic of China (ROC)', "Taiwan")
df_org = df_org.replace('United Kingdom of Great Britain & Northern Ireland', 'United Kingdom')
df_org = df_org.replace('EMPTY', 'Unknown')

In [ ]:
def getIndexes(dfObj, value):
  #https://thispointer.com/python-find-indexes-of-an-element-in-pandas-dataframe/
    ''' Get index positions of value in dataframe i.e. dfObj.'''
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos


In [ ]:
listOfPositions = getIndexes(cont_simple, 'Albania')
for i in range(len(listOfPositions)):
  print('Position ', i, ' (Row index , Column Name) : ', cont_simple['region'][listOfPositions[i][0]])

Position  0  (Row index , Column Name) :  Europe


In [ ]:
df = df_org

In [ ]:
df['region'] = "Hi"

In [ ]:
df.head()

,country,year,parent_company,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers,region
0,Argentina,2019,Grand Total,0.0,215.0,55.0,607.0,1376.0,281.0,116.0,18.0,2668.0,4,243.0,Hi
1,Argentina,2019,Unbranded,0.0,155.0,50.0,532.0,848.0,122.0,114.0,17.0,1838.0,4,243.0,Hi
2,Argentina,2019,The Coca-Cola Company,0.0,0.0,0.0,0.0,222.0,35.0,0.0,0.0,257.0,4,243.0,Hi
3,Argentina,2019,Secco,0.0,0.0,0.0,0.0,39.0,4.0,0.0,0.0,43.0,4,243.0,Hi
4,Argentina,2019,Doble Cola,0.0,0.0,0.0,0.0,38.0,0.0,0.0,0.0,38.0,4,243.0,Hi


In [ ]:
def convert_country(row):
  listOfPositions = getIndexes(cont_simple, row['country'])
  if len(listOfPositions) > 0:
    return cont_simple['region'][listOfPositions[0][0]]
  else:
    return np.nan
  

In [ ]:
df['region'] = df.apply(lambda row: convert_country(row), axis=1)

In [ ]:
df[df['region'].isna()].groupby('country').count()

,year,parent_company,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers,region
country,,,,,,,,,,,,,,
Unknown,1346,1346,0,1346,1346,1346,1346,1346,1346,1346,1346,1346,1346,0


# General understanding of data

In [ ]:
df.shape

(13380, 15)

In [ ]:
for each in df.columns:
  print(type(each))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [ ]:
df[["year", "empty", "hdpe", "ldpe", "o", "pp", "ps", "pvc", "grand_total", "num_events", "volunteers"]] = df[["year", "empty", "hdpe", "ldpe", "o", "pp", "ps", "pvc", "grand_total", "num_events", "volunteers"]].apply(pd.to_numeric)

In [ ]:
print(df.dtypes)

country            object
year                int64
parent_company     object
empty             float64
hdpe              float64
ldpe              float64
o                 float64
pet               float64
pp                float64
ps                float64
pvc               float64
grand_total       float64
num_events          int64
volunteers        float64
region             object
dtype: object


In [ ]:
df.groupby("year")["country"].count()

year
2019    9296
2020    4084
Name: country, dtype: int64

# Replace NA values

In [ ]:
#df[df["parent_company"].isna()]
#for col in df.columns:
  #print(df[df[col].isna()])
df["parent_company"] = df["parent_company"].fillna("Unbranded")
df["empty"] = df["empty"].fillna(0.0)
df["hdpe"] = df["hdpe"].fillna(0.0)
df["ldpe"] = df["ldpe"].fillna(0.0)
df["o"] = df["o"].fillna(0.0)
df["pet"] = df["pet"].fillna(0.0)
df["pp"] = df["pp"].fillna(0.0)
df["ps"] = df["ps"].fillna(0.0)
df["pvc"] = df["ps"].fillna(0.0)
df["grand_total"] = df["grand_total"].fillna(0.0)
df["num_events"] = df["num_events"].fillna(0)
df["volunteers"] = df["volunteers"].fillna(0.0)


# Checking country and company sizes

In [ ]:
df.groupby("country").size().sort_values(ascending = False)

country
Unknown                     1346
United States of America     959
Indonesia                    930
China                        838
Philippines                  831
                            ... 
Benin                          4
Luxembourg                     2
Honduras                       2
Côte D'Ivoire                  2
Armenia                        2
Length: 66, dtype: int64

In [ ]:
company_groups = df.groupby(['parent_company','country']).count().reset_index()
company_groups[company_groups['year'] > 2]
df[(df['parent_company'] == 'Unbranded') &(df['country'] == 'Argentina')]

,country,year,parent_company,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers,region
1,Argentina,2019,Unbranded,0.0,155.0,50.0,532.0,848.0,122.0,114.0,114.0,1838.0,4,243.0,Americas
9296,Argentina,2020,Unbranded,0.0,12.0,9.0,70.0,9.0,7.0,2.0,2.0,109.0,24,9.0,Americas
9508,Argentina,2020,Unbranded,126.0,80.0,52.0,6.0,46.0,58.0,20.0,20.0,388.0,24,9.0,Americas


# Create 2019 and 2020 only df

In [ ]:
df2019 = df[df['year'] == 2019]
df2020 = df[df['year'] == 2020]

In [ ]:
df.groupby('year').sum().reset_index()

,year,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers
0,2019,0.0,18640.0,79178.0,518138.0,193796.0,35976.0,11780.0,11780.0,858462.0,322896,12710669.0
1,2020,4174.0,17102.0,37462.0,132445.0,81904.0,61720.0,9463.0,9463.0,346494.0,123592,2123837.0


# Region info

In [ ]:
reg_year_df = df.groupby(['region', 'year']).sum().reset_index()
reg_year_df

,region,year,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers
0,Africa,2019,0.0,9200.0,34698.0,61656.0,116916.0,2464.0,404.0,404.0,225434.0,10229,890225.0
1,Africa,2020,335.0,8718.0,16460.0,26764.0,54696.0,3292.0,2158.0,2158.0,112496.0,6559,795810.0
2,Americas,2019,0.0,596.0,492.0,35044.0,3988.0,11234.0,462.0,462.0,51852.0,27942,600745.0
3,Americas,2020,671.0,1747.0,3353.0,8560.0,9089.0,6243.0,1721.0,1721.0,31733.0,58623,211116.0
4,Asia,2019,0.0,5354.0,30728.0,314806.0,41086.0,9472.0,6224.0,6224.0,408102.0,80689,8997499.0
5,Asia,2020,697.0,2977.0,12513.0,27447.0,12542.0,47374.0,2834.0,2834.0,107948.0,47960,936223.0
6,Europe,2019,0.0,420.0,374.0,46270.0,9626.0,1966.0,488.0,488.0,59158.0,8864,316254.0
7,Europe,2020,2467.0,3627.0,4914.0,68776.0,5436.0,4450.0,2747.0,2747.0,92649.0,9829,167578.0
8,Oceania,2019,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,6.0,2,10.0
9,Oceania,2020,4.0,33.0,222.0,898.0,141.0,361.0,3.0,3.0,1668.0,621,13110.0


In [ ]:
total_plastic = df.num_events.sum()
total_plastic

446488

In [ ]:
reg_year_df['percentTotal'] = reg_year_df.grand_total.apply(lambda x: x/total_plastic)

In [ ]:
total_region = df.groupby('region').grand_total.sum()
total_region = total_region.to_frame().reset_index()
total_region

,region,grand_total
0,Africa,337930.0
1,Americas,83585.0
2,Asia,516050.0
3,Europe,151807.0
4,Oceania,1674.0


In [ ]:
countryAndCompany = df.groupby(['country', 'parent_company']).sum().reset_index()
countryAndCompany[countryAndCompany['country']=='United States of America'].sort_values(by = 'grand_total', ascending=False).head()

,country,parent_company,year,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers
10647,United States of America,Unbranded,6059,322.0,331.0,501.0,3891.0,1574.0,5298.0,635.0,635.0,12685.0,310,1762.0
10162,United States of America,Grand Total,2019,0.0,20.0,169.0,975.0,169.0,2913.0,68.0,68.0,4314.0,42,740.0
10613,United States of America,The Kroger Company,4039,0.0,97.0,27.0,9.0,49.0,309.0,10.0,10.0,501.0,176,1251.0
10434,United States of America,Pepsico,2020,9.0,1.0,32.0,74.0,300.0,18.0,1.0,1.0,440.0,134,511.0
10610,United States of America,The Coca-Cola Company,4039,6.0,0.0,3.0,21.0,221.0,27.0,5.0,5.0,283.0,176,1251.0


# Brands

In [ ]:
df_brands = df[(df['parent_company'] !='Grand Total')] 
df_brands = df_brands[df_brands['parent_company'] !='Unbranded']
df_brands = df_brands[df_brands['parent_company'] !='Assorted']

In [ ]:
df_brands[df_brands['parent_company'].str.contains("cigar")].groupby('parent_company').count()

,country,year,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers,region
parent_company,,,,,,,,,,,,,,
Sportsman cigarret,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
df_brands = df_brands.replace(['Nestlé', 'Aguas CCU- Nestle', 'CCU - Nestle', 'Maggi(Nestle)', 'Nestle Granola', 'Nestle Product', 'Nestle Product Sdn. Bhd.', 'Nestles: Smarties', 'NestlÃ¨', 'Uncle Tobys Nestle', 'Vera NestlÃ¨', 'nestle drinking water', 'nestlÃ¨', 'Gerber Product Company', 'Nesquik', 'Cailler', 'Kitkat Matcha', 'Orion', 'Orion Confectionery', 'nescafe classic', 'Coffee-mate', 'Haagen-Dazs', 'Hagen', 'Purina', 'Purina gourmet', 'Mövenpick'], 'Nestle')
df_brands = df_brands.replace(['Hindustan Unilever', 'Axe', 'Clear', 'Comfort', 'Dove', 'Hellmanns', 'Hellmans', 'Knorr', 'Magnum', 'Omo', 'M/s Radiant Indus Chem Pvt. Ltd /Foodrite India', 'M/s Radiant Indus Chem Pvt. Ltd.', 'Smile', 'Smile (Тов Кпд)', 'Тов Кпд Biosphere Smile', 'Sunlight Resources', 'Sunlight Resources Ltd', 'Sunlite', 'Sun light', 'Surf Fabcon', 'Surf Fabric Conditioner', 'Ala', 'Marca Maizena', 'Persil', 'Pond', "Pond'S", 'Rinso Sachet', 'Rinso sachet', 'Suave', 'Solero Exotic', 'Surf Fabcon', 'Surf Fabric Conditioner', 'VO5', "Wall'S", "Wall's", 'Walls', 'Clinic Plus'], 'Unilever')
df_brands = df_brands.replace(['URC',"Jack 'n Jill", "Jack n' Jill", 'Calbee Inc.', 'Conagra', 'Conagra Brands', 'Aqua Danone', "Aqua d’Or Mineral Water a/s / Danone", 'Danone', 'Danone Foods Ltd', 'Grupo Danone', 'Простоквашино(Danone)', 'El Real Spaghetti 400gm', 'Nissin', 'Nissin Cup Noodles', 'Vitasoy', 'Vitasoy International Holdings', 'Vitasoy International Holdings Limited'], 'Universal Robina Corporation')
df_brands = df_brands.replace(['Philip Morris International', 'Chesterfield (Philip Morris)', 'Heets (Philip Morris)', 'L&M (Philip Morris)', 'Philip Meris', 'Marlboro', 'Marlboro Cigarettes'], 'Philip Morris')
df_brands = df_brands.replace(['P&G', 'p & g'], 'Procter & Gamble')
df_brands = df_brands.replace(['Pepsico', 'Frito-Lay', 'FritoLay', 'Frito_Lay', 'Fruit Lay', 'Lay', "Lay'S", 'Lays Chip', 'Lays Chips', 'Lays chip bag', 'Lays chips', 'Patatas Lays', 'Doritos Bits','Monster, Coors, Modelo, Blue Sky, Minute Maid, Diet Samâ€™s Cola, Budweiser, Mtn Dew', 'Dasani, Gatorade, Unknown', 'Gatorade', 'Tropicano', 'Starbucks Coffee', 'Starbucks', 'Starbucks Straws', 'Fritito', 'Frito Chips', 'Seven Up', 'Mirinda', 'Mirinda Orange'], 'PepsiCo')
df_brands = df_brands.replace('Etika - PepsiCo', 'PepsiCo')
df_brands = df_brands.replace(['Colgate', 'Speed Stick', "Tom's of Maine", 'Ajax'], 'Colgate-Palmolive')
df_brands = df_brands.replace('Flag; Fanta; Faxe; Inconnu', 'Inconnu')
df_brands = df_brands.replace(['PT Coca Cola Indonesia', 'Innocent / The Coca-Cola Company', 'Fanta', 'FantaSea', 'Pineapple Fanta', 'Smart water', 'Minute Maid (Bottle)', 'innocent', 'Innocent', 'Costa', 'Fuzetea', 'Honest Kids', 'Honest kids', 'GlacÃ©au Vitaminwater', 'Topo Chico','Peace'], 'The Coca-Cola Company')
df_brands = df_brands.replace(['Universal Robina', 'Universal Robina Corp'], 'Universal Robina Corporation')
df_brands = df_brands.replace(['Mondeles', 'Mondelez Kinh Do Viet Nam Company', 'MondelÄ“z Japan', 'Moreni waffle - Mondelez Iternational', 'Teddy Grahams (Mondelez Global)', 'Tuc (Mondelez)', 'Барні, Mondelēz International', 'Люкс, Mondelēz International'], 'Mondelez International')
df_brands = df_brands.replace(['Bounty (Mars)', 'Bounty Mars Украина', 'M&M/Mars', 'Mars', 'Mars Inc', 'Mars Incorporated', 'Mars International', 'Snickers (Mars)', 'Snickers Mars Украина', 'Twix Mars'], 'Mars, Incorporated')
df_brands = df_brands.replace(['Perfetti van Melle', 'Perfetti Van', 'Perfetti Melle', 'Perfelle Van Melle', 'Perfectti Van Mele Bangladesh Private ltd.', 'Perfette', 'Perfetti', 'Mentos Gum', 'Mentos Mint', 'Alpenliebe', 'Airheads Xtream', 'Center fresh gum', 'Fruitella'], 'Perfetti Van Melle')

In [ ]:
df_brands[(df_brands['parent_company']!='La Doo') & (df_brands['parent_company']!='Barna')].groupby('parent_company').sum().reset_index().sort_values(by='grand_total', ascending = False).head(10)

,parent_company,year,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers
8626,The Coca-Cola Company,214068,23.0,1611.0,461.0,1283.0,20801.0,1309.0,52.0,52.0,25640.0,2327,60324.0
6010,Nestle,212045,215.0,233.0,1023.0,3989.0,1667.0,6256.0,102.0,102.0,13622.0,1838,58075.0
9032,Universal Robina Corporation,173679,2.0,186.0,201.0,2155.0,2313.0,6543.0,72.0,72.0,11523.0,2622,78475.0
6665,PepsiCo,232244,21.0,277.0,272.0,1689.0,5496.0,989.0,76.0,76.0,8863.0,2618,60397.0
9020,Unilever,199927,5.0,355.0,1164.0,3402.0,473.0,3317.0,3.0,3.0,8738.0,2283,85461.0
1807,Colgate-Palmolive,96940,4.0,135.0,35.0,694.0,54.0,5617.0,6.0,6.0,6552.0,1155,29540.0
7030,"Pure Water, Inc.",8077,0.0,0.0,321.0,0.0,5026.0,0.0,0.0,0.0,5347.0,44,8064.0
6709,Philip Morris,109056,33.0,13.0,172.0,4534.0,3.0,88.0,4.0,4.0,4848.0,923,67623.0
6942,Procter & Gamble,98960,8.0,87.0,221.0,1420.0,231.0,2603.0,5.0,5.0,4616.0,929,33181.0
5494,Mayora Indah,18180,2.0,84.0,3.0,109.0,275.0,4026.0,0.0,0.0,4499.0,201,3287.0


In [ ]:
df_brands[(df_brands['year'] == 2019) & (df_brands['parent_company']!='La Doo') & (df_brands['parent_company']!='Barna')].groupby('parent_company').sum().reset_index().sort_values(by='grand_total', ascending = False).head(10)

,parent_company,year,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers
6398,The Coca-Cola Company,104988,0.0,877.0,97.0,978.0,9258.0,454.0,44.0,44.0,11759.0,1678,44874.0
5201,"Pure Water, Inc.",6057,0.0,0.0,321.0,0.0,4989.0,0.0,0.0,0.0,5310.0,35,7727.0
4513,Nestle,111045,0.0,178.0,803.0,2475.0,1113.0,193.0,84.0,84.0,4922.0,1085,46395.0
4928,PepsiCo,113064,0.0,3.0,32.0,995.0,2066.0,390.0,38.0,38.0,3525.0,1442,41379.0
6690,Universal Robina Corporation,82779,0.0,62.0,179.0,1766.0,1332.0,92.0,14.0,14.0,3445.0,1352,62533.0
6681,Unilever,107007,0.0,63.0,1009.0,1726.0,137.0,208.0,1.0,1.0,3144.0,1249,64046.0
3035,Inconnu,4038,0.0,0.0,2057.0,254.0,602.0,113.0,27.0,27.0,3053.0,2,470.0
4955,Philip Morris,48456,0.0,13.0,47.0,2186.0,1.0,0.0,4.0,4.0,2251.0,408,60591.0
5345,Reckitt Benckiser,24228,0.0,994.0,6.0,609.0,11.0,12.0,0.0,0.0,1632.0,213,8763.0
5596,Sahakari Jal,2019,0.0,0.0,0.0,0.0,563.0,975.0,0.0,0.0,1538.0,145,1416.0


In [ ]:
df_brands[(df_brands['year'] == 2020) & (df_brands['parent_company']!='La Doo') & (df_brands['parent_company']!='Barna')].groupby('parent_company').sum().reset_index().sort_values(by='grand_total', ascending = False).head(10)

,parent_company,year,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers
2576,The Coca-Cola Company,109080,23.0,734.0,364.0,305.0,11543.0,855.0,8.0,8.0,13881.0,649,15450.0
1733,Nestle,101000,215.0,55.0,220.0,1514.0,554.0,6063.0,18.0,18.0,8700.0,753,11680.0
2707,Universal Robina Corporation,90900,2.0,124.0,22.0,389.0,981.0,6451.0,58.0,58.0,8078.0,1270,15942.0
544,Colgate-Palmolive,56560,4.0,118.0,9.0,157.0,46.0,5574.0,2.0,2.0,5917.0,817,9329.0
2703,Unilever,92920,5.0,292.0,155.0,1676.0,336.0,3109.0,2.0,2.0,5594.0,1034,21415.0
2000,PepsiCo,119180,21.0,274.0,240.0,694.0,3430.0,599.0,38.0,38.0,5338.0,1176,19018.0
1579,Mayora Indah,18180,2.0,84.0,3.0,109.0,275.0,4026.0,0.0,0.0,4499.0,201,3287.0
2536,Tamil Nadu Co-operative Milk Producers' Federa...,2020,0.0,0.0,4229.0,10.0,9.0,2.0,0.0,0.0,4250.0,56,353.0
2082,Procter & Gamble,58580,8.0,75.0,65.0,602.0,184.0,2575.0,4.0,4.0,3535.0,514,9201.0
2023,Philip Morris,60600,33.0,0.0,125.0,2348.0,2.0,88.0,0.0,0.0,2597.0,515,7032.0


In [ ]:
df_brands[df_brands['year'] == 2019].groupby('parent_company').count().country.sort_values(ascending = False).head(10)

parent_company
PepsiCo                         56
Nestle                          55
Unilever                        53
The Coca-Cola Company           52
Universal Robina Corporation    41
Mondelez International          33
Mars, Incorporated              32
Perfetti Van Melle              30
Philip Morris                   24
Colgate-Palmolive               20
Name: country, dtype: int64

In [ ]:
df_brands[df_brands['year'] == 2020].groupby('parent_company').count().country.sort_values(ascending = False).head(10)

parent_company
PepsiCo                         59
The Coca-Cola Company           54
Nestle                          50
Unilever                        46
Universal Robina Corporation    45
Mars, Incorporated              34
Mondelez International          34
Philip Morris                   30
Procter & Gamble                29
Colgate-Palmolive               28
Name: country, dtype: int64

In [ ]:
df_brands[df_brands['parent_company'] == 'Reckitt Benckiser']

,country,year,parent_company,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers,region
617,Cameroon,2019,Reckitt Benckiser,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,6.0,10,387.0,Africa
1609,Unknown,2019,Reckitt Benckiser,0.0,0.0,0.0,16.0,1.0,0.0,0.0,0.0,17.0,145,1416.0,NaN
3390,India,2019,Reckitt Benckiser,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,4,88.0,Asia
4209,Ireland,2019,Reckitt Benckiser,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,Europe
4237,Italy,2019,Reckitt Benckiser,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,175.0,Europe
4705,Latvia,2019,Reckitt Benckiser,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,1,13.0,Europe
5169,Nigeria,2019,Reckitt Benckiser,0.0,985.0,0.0,570.0,10.0,0.0,0.0,0.0,1565.0,14,1648.0,Africa
5586,Netherlands,2019,Reckitt Benckiser,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,8.0,2,827.0,Europe
5952,Philippines,2019,Reckitt Benckiser,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,20,3751.0,Asia
6422,Slovenia,2019,Reckitt Benckiser,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,12.0,1,9.0,Europe


In [ ]:
df_brands.to_csv('PlasticCleaned.csv')

In [ ]:
df_brands[df_brands['region'] == 'Asia'].groupby('country').sum()

,year,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers
country,,,,,,,,,,,,
Bangladesh,333276,0.0,374.0,2.0,1575.0,282.0,89.0,36.0,36.0,2358.0,165,20307.0
Bhutan,6057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,15900.0
China,1690119,0.0,2.0,0.0,4480.0,1309.0,0.0,0.0,0.0,5791.0,30024,1363419.0
Cyprus,96912,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,24.0,48,1440.0
Hong Kong,1122564,0.0,244.0,7.0,782.0,1909.0,116.0,4.0,4.0,3062.0,3892,37252.0
India,1116853,4.0,419.0,6391.0,6370.0,546.0,658.0,17.0,17.0,14478.0,20204,140354.0
Indonesia,1869886,4.0,720.0,1629.0,2823.0,5191.0,2316.0,131.0,131.0,12843.0,34888,4457364.0
Japan,428031,0.0,0.0,6.0,20.0,7.0,13.0,2.0,2.0,53.0,7318,270246.0
Korea,212100,1.0,5.0,100.0,873.0,48.0,93.0,39.0,39.0,1167.0,2730,0.0


In [ ]:
df_brands.groupby('parent_company').sum().sort_values(by='pvc', ascending = False)

,year,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers
parent_company,,,,,,,,,,,,
McDonald's Corporation,32304,0.0,13.0,24.0,591.0,9.0,143.0,182.0,182.0,962.0,285,9741.0
Indofood,24234,0.0,49.0,620.0,169.0,767.0,269.0,110.0,110.0,2009.0,359,14028.0
Nestle,212045,215.0,233.0,1023.0,3989.0,1667.0,6256.0,102.0,102.0,13622.0,1838,58075.0
Malindu,2019,0.0,0.0,0.0,0.0,7.0,1.0,81.0,81.0,89.0,2,48.0
PepsiCo,232244,21.0,277.0,272.0,1689.0,5496.0,989.0,76.0,76.0,8863.0,2618,60397.0
...,...,...,...,...,...,...,...,...,...,...,...,...
Haiter,2019,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5,374.0
Haitian,2019,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,48,1439.0
Haitian-Food,2019,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,48,1439.0


# Total Plastic

In [ ]:
total_count = df[(df['parent_company'] !='Grand Total')]

In [ ]:
total_count[total_count['year']==2019].groupby('country').sum().sort_values(by='grand_total', ascending = False)

,year,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers
country,,,,,,,,,,,,
Taiwan,20190,0.0,0.0,0.0,120646.0,0.0,0.0,0.0,0.0,120646.0,20,313180.0
Nigeria,712707,0.0,3728.0,11700.0,28055.0,36226.0,708.0,120.0,120.0,80570.0,4942,581744.0
Unknown,2715555,0.0,1535.0,6443.0,30181.0,11087.0,5420.0,2101.0,2101.0,56955.0,195025,1904520.0
Philippines,1433490,0.0,1439.0,8671.0,19841.0,4906.0,1305.0,766.0,766.0,37016.0,14200,2663210.0
Indonesia,1282065,0.0,195.0,2798.0,4181.0,3558.0,1188.0,1365.0,1365.0,13309.0,20320,4349750.0
Ecuador,98931,0.0,0.0,0.0,11186.0,54.0,1475.0,0.0,0.0,12715.0,49,22295.0
Vietnam,1366863,0.0,29.0,2101.0,4970.0,2986.0,408.0,350.0,350.0,10887.0,2708,270800.0
Kenya,211995,0.0,215.0,135.0,105.0,8892.0,127.0,20.0,20.0,9494.0,525,163800.0
Cameroon,304869,0.0,595.0,3425.0,1009.0,3263.0,284.0,10.0,10.0,8595.0,1510,58437.0


In [ ]:
total_unbranded = total_count[total_count['parent_company'] == 'Unbranded'].grand_total.sum()

In [ ]:
(1 - total_unbranded / total_count.grand_total.sum()) * 100

39.63517999290986

In [ ]:
total_count.groupby('country').count().reset_index().country

0                    Argentina
1                      Armenia
2                    Australia
3                   Bangladesh
4                        Benin
                ...           
61        United Arab Emirates
62              United Kingdom
63    United States of America
64                     Unknown
65                     Vietnam
Name: country, Length: 66, dtype: object

In [ ]:
total_count.to_csv('TotalPlasticCount.csv')

# Checking top companies plastic counts


In [ ]:
coke_trend = df_brands[df_brands['parent_company'] =="The Coca-Cola Company"].groupby('year').sum().reset_index()
coke_trend

,year,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers
0,2019,0.0,877.0,97.0,978.0,9258.0,454.0,44.0,44.0,11759.0,1678,44874.0
1,2020,23.0,734.0,364.0,305.0,11543.0,855.0,8.0,8.0,13881.0,649,15450.0


In [ ]:
nestle_trend = df_brands[df_brands['parent_company'] =="Nestle"].groupby('year').sum().reset_index()
nestle_trend

,year,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers
0,2019,0.0,178.0,803.0,2475.0,1113.0,193.0,84.0,84.0,4922.0,1085,46395.0
1,2020,215.0,55.0,220.0,1514.0,554.0,6063.0,18.0,18.0,8700.0,753,11680.0


In [ ]:
robina_trend = df_brands[df_brands['parent_company'] =="Universal Robina Corporation"].groupby('year').sum().reset_index()
robina_trend

,year,empty,hdpe,ldpe,o,pet,pp,ps,pvc,grand_total,num_events,volunteers
0,2019,0.0,62.0,179.0,1766.0,1332.0,92.0,14.0,14.0,3445.0,1352,62533.0
1,2020,2.0,124.0,22.0,389.0,981.0,6451.0,58.0,58.0,8078.0,1270,15942.0


In [ ]:
fig = go.Figure(data=[
    go.Bar(name='The Coca-Cola Company', x=coke_trend.year, y=coke_trend.grand_total),
    go.Bar(name='Nestle', x=nestle_trend.year, y=nestle_trend.grand_total),
    go.Bar(name='Universal Robina Corporation', x=robina_trend.year, y=robina_trend.grand_total)
])

fig.update_layout(barmode='group')
fig.update_layout(title = 'Top Plastic Pollution Brands Saw an Increase of Plastic from 2019 to 2020', xaxis_title = 'Year', yaxis_title = 'Total Plastic Count')
fig.show()
